In [1]:
# Import Dependencies
import pandas as pd
import numpy as np

In [2]:
# Create a path to the JSON and read it into a Pandas DataFrame
json_path = "purchase_data.json"
purchase_data_df = pd.read_json(json_path, orient = "records")

purchase_data_df.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [3]:
#Total Players
player_demos = purchase_data_df.loc[:,["SN","Gender","Age"]]
#Drop Duplicates
player_demos = player_demos.drop_duplicates()
total_players= player_demos.count()[0]
total_players_df = pd.DataFrame({"Total Players":[total_players]})
total_players_df

,Total Players
0,573


In [4]:
#Purchasing Analysis (Total)

item_count = len(purchase_data_df["Item ID"].unique())

#Calculate the Average Purchase Price
avg_price = purchase_data_df["Price"].mean()
#Calculate Total Number of Purchases
tot_purchases = purchase_data_df["Item ID"].count()
tot_revenue = purchase_data_df["Price"].sum()
purchasing_analysis = pd.DataFrame({"Total Unique Items":[item_count],
                                    "Avg Purchase Price":[avg_price],
                                    "Total Purchases":[tot_purchases],
                                    "Total Revenue":[tot_revenue]})
purchasing_analysis["Avg Purchase Price"] = purchasing_analysis["Avg Purchase Price"].map("${:.2f}".format)
purchasing_analysis["Total Revenue"] = purchasing_analysis["Total Revenue"].map("${:.2f}".format)
ord_purchasing_analysis = purchasing_analysis[["Total Unique Items","Avg Purchase Price","Total Purchases","Total Revenue"]]
ord_purchasing_analysis

,Total Unique Items,Avg Purchase Price,Total Purchases,Total Revenue
0,183,$2.93,780,$2286.33


In [80]:
#Gender Demographics
gender_counts = player_demos["Gender"].value_counts().reset_index()
gender_counts.columns = ["Gender","Total Count"]
percent = (gender_counts["Total Count"]/total_players)* 100
gender_counts["Percent of Players"]= percent
ord_gender_counts = gender_counts[["Gender","Percent of Players","Total Count"]]
ord_gender_counts

,Gender,Percent of Players,Total Count
0,Male,81.151832,465
1,Female,17.452007,100
2,Other / Non-Disclosed,1.396161,8


In [108]:
#Purchasing Analysis(Gender)
#player_analysis= purchase_data_df["Gender"].value_counts().reset_index()
#player_analysis.columns = ["Gender","Purchase Count"]
grouped_purchase_data = purchase_data_df.groupby(["Gender"])
grouped_purchase_data_df = pd.DataFrame(grouped_purchase_data.count())


price_grouped = grouped_purchase_data["Price"].sum().reset_index()

total_purchase_value = price_grouped["Price"]
print(total_purchase_value)

grouped_purchase_data_df["Total Purchase Value"] = total_purchase_value





0     382.91
1    1867.68
2      35.74
Name: Price, dtype: float64


,SN
Gender,
Female,100
Male,465
Other / Non-Disclosed,8
